<center><img src="https://unskript.com/assets/favicon.png" alt="unSkript.com" width="100" height="100"/> 
<h1> unSkript Runbooks </h1>
<div class="alert alert-block alert-success">
    <h3> Objective</h3> <br>
    <b style = "color:#000000"><i>Renew expiring AWS Certificate Manager(ACM) issued SSL Certificates</i></b>
</div>
</center>
<br>

<center><h2><u>Renew SSL Certificate</u></h2></center>

# Steps Overview
1)[ List expiring ACM certificates](#1)<br>
2)[ Renew expiring ACM certificates](#2)

<h3><a id="1">List all AWS Regions</a></h3>
This action fetches all AWS Regions to execute Step 1👇. This action will only execute if no <code>region</code> is provided.

>This action takes the following parameters: `None`

>This action captures the following ouput: `all_regions`

In [11]:
#
# Copyright (c) 2021 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field
from typing import Dict, List
import pprint

from beartype import beartype
@beartype
def aws_list_all_regions_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_list_all_regions(handle) -> List:
    """aws_list_all_regions lists all the AWS regions

        :type handle: object
        :param handle: Object returned from Task Validate

        :rtype: Result List of result
    """

    result = handle.aws_cli_command("aws ec2 describe-regions --all-regions --query 'Regions[].{Name:RegionName}' --output text")
    if result is None or result.returncode != 0:
        print("Error while executing command : {}".format(result))
        return str()
    result_op = list(result.stdout.split("\n"))
    list_region = [x for x in result_op if x != '']
    return list_region


task = Task(Workflow())
task.configure(outputName="all_regions")
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_list_all_regions, lego_printer=aws_list_all_regions_printer, hdl=hdl, args=args)

<h3><a id="1">List expiring ACM certificates</a></h3>
Using unSkript's List expiring ACM certificates action, we will fetch all the expiring certificates given a specific number of threshold days.

>This action takes the following parameters: `threshold_days`, `region`

>This action captures the following output: `all_certs`

In [12]:
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
import dateutil
from pydantic import BaseModel, Field
from typing import Dict,List
import pprint
import datetime

from beartype import beartype
@beartype
def aws_list_expiring_acm_certificates_printer(output):
    if output is None:
        return
    pprint.pprint(output)

@beartype
def aws_list_expiring_acm_certificates(handle, threshold_days: int, region: str='')-> Dict:
    """aws_list_expiring_acm_certificates returns all the ACM issued certificates which are about to expire given a threshold number of days

        :type handle: object
        :param handle: Object returned from Task Validate

        :type threshold_days: int
        :param threshold_days: Threshold number of days to check for expiry. Eg: 30 -lists all certificates which are expiring within 30 days

        :type region: str
        :param region: Region name of the AWS account

        :rtype: Result Dictionary of result
    """
    iamClient = handle.client('acm', region_name=region)
    arn_list=[]
    domain_list = []
    expiring_certificates_list= []
    expiring_certificates_dict={}
    result_list=[]
    try:
        certificates_list = iamClient.list_certificates(CertificateStatuses=['ISSUED'])
        for each_arn in certificates_list['CertificateSummaryList']:
            arn_list.append(each_arn['CertificateArn'])
            domain_list.append(each_arn['DomainName'])
        for cert_arn in arn_list:
            details = iamClient.describe_certificate(CertificateArn=cert_arn)
            for key,value in details['Certificate'].items():
                if key == "NotAfter":
                    expiry_date = value
                    right_now = datetime.datetime.now(dateutil.tz.tzlocal())
                    diff = expiry_date-right_now
                    days_remaining = diff.days
                    if days_remaining < threshold_days and days_remaining > 0:
                        expiring_certificates_list.append(cert_arn)
                    expiring_certificates_dict["region"]= region
                    expiring_certificates_dict["certificate"]= expiring_certificates_list
        result_list.append(expiring_certificates_dict)
    except Exception as e:
        pass
    return expiring_certificates_dict


task = Task(Workflow())
task.configure(printOutput=False)
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "threshold_days": "threshold_days",
    "region": "iter_item"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "all_regions",
    "iter_parameter": "region"
    }''')
task.configure(outputName="all_certs")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_list_expiring_acm_certificates, lego_printer=aws_list_expiring_acm_certificates_printer, hdl=hdl, args=args)

<h3><a id="1">Create List of Expiring Certificates</a></h3>
This action filters regions that have no certificates and creates a list of certificates that have to be renewed

>This action takes the following parameters: `None`

>This action captures the following output: `expiring_certificates`

In [13]:
def printer():
    result=[]
    for k,v in all_certs.items():
        if len(v)!=0:
            result.append(v)
    print(result)
    return result

expiring_certificates= printer()

<h3><a id="2">Renew expiring ACM certificates</a></h3>
This action renews <b>eligible</b> SSL certificates that are available on ACM. Only exported private certificates can be renewed with this operation. In order to renew your AWS Private CA certificates with ACM, you must first grant the <a href="https://docs.aws.amazon.com/privateca/latest/userguide/PcaWelcome.html">ACM service principal permission</a> to do so.<br><br>
<i><b>A certificate is eligible for automatic renewal subject to the following considerations:</b></i><br>


1)<span style="color:green"> ELIGIBLE</span> if associated with another AWS service, such as Elastic Load Balancing or CloudFront.<br>
2)<span style="color:green"> ELIGIBLE</span>if exported since being issued or last renewed.<br>
3)<span style="color:green"> ELIGIBLE</span> if it is a private certificate issued by calling the ACM RequestCertificate API and then exported or associated with another AWS service.<br>
4)<span style="color:green"> ELIGIBLE</span> if it is a private certificate issued through the management console and then exported or associated with another AWS service.<br>
5)<span style="color:red"> NOT ELIGIBLE</span> if it is a private certificate issued by calling the AWS Private CA IssueCertificate API.<br>
6)<span style="color:red"> NOT ELIGIBLE</span> if imported or already expired<br>

>This action takes the following parameters: `aws_certificate_arn`, `region`

>This action captures the following output: `renewed_certificates`

In [ ]:
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
import dateutil
from pydantic import BaseModel, Field
from typing import Dict,List
import pprint
import datetime

from beartype import beartype
@beartype
def aws_renew_expiring_acm_certificates_printer(output):
    if output is None:
        return
    pprint.pprint(output)

@beartype
def aws_renew_expiring_acm_certificates(handle, aws_certificate_arn: List, region: str='') -> Dict:
    """aws_renew_expiring_acm_certificates returns all the ACM issued certificates which are about to expire given a threshold number of days

        :type handle: object
        :param handle: Object returned from Task Validate

        :type aws_certificate_arn: List
        :param aws_certificate_arn: ARN of the Certificate

        :type region: str
        :param region: Region name of the AWS account

        :rtype: Result Dictionary of result
    """
    result = {}
    try:
        acmClient = handle.client('acm', region_name=region)
        for arn in aws_certificate_arn:
            acmClient.renew_certificate(CertificateArn=arn)
            result[arn] = "Successfully renewed"
    except Exception as e:
        result["error"] = e
    return result


task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "aws_certificate_arn": "iter.get(\\"certificate\\")",
    "region": "iter.get(\\"region\\")"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "expiring_certificates",
    "iter_parameter": ["aws_certificate_arn","region"]
    }''')

task.configure(outputName="renewed_certificates")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_renew_expiring_acm_certificates, lego_printer=aws_renew_expiring_acm_certificates_printer, hdl=hdl, args=args)

<h3>Conclusion</h3> 
<p>In this Runbook, we demonstrated the use of unSkript's AWS actions to list all expiring ACM SSL Certificates and subsequently renewed them. To view the full platform capabilities of unSkript please visit
<a href="https://us.app.unskript.io">unskript.com</a></p>